TODO: add some preprocessing of the data: remove punctuation

In [37]:
def compute_metrics(predicted_labels, golden_labels):
    # Initialize counters for full matches and total pairs
    exact_matches = 0
    total_pairs = len(predicted_labels)

    # Iterate over each pair of predicted and golden labels
    for predicted, golden in zip(predicted_labels, golden_labels):
        # Compare the phrases after converting to lowercase
        if predicted.lower() == golden.lower():
            exact_matches += 1

    # Calculate Precision, Recall, and F1 Score
    precision = exact_matches / total_pairs if total_pairs > 0 else 0
    recall = exact_matches / total_pairs if total_pairs > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

In [42]:
predicted_labels = ["Appeal to Tradition", "appeal to fear"]
golden_labels = ["appeal to fear", "appeal to Fear"]

precision, recall, f1_score = compute_metrics(predicted_labels, golden_labels)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Precision: 0.5
Recall: 0.5
F1 Score: 0.5
